In [1]:
!pip install keras_transformer
import numpy as np
import matplotlib.pyplot as plt
from keras_transformer import get_model, decode
data_dir = "drive/My Drive/CS4995 Deep Learning/Competitions_Project/Final_Project/data/3000/new/"

Using TensorFlow backend.


In [0]:
#load the pairs list we have dumped with pickle package
import pickle
pickle_in = open(data_dir+"dict.pickle","rb")
data = pickle.load(pickle_in)

In [0]:
import numpy as np
from keras_transformer import get_model, decode

source_tokens = [item[0].split(',') for item in data]  #source_tokens[0] = ['AAAA', 'BBBB',...,'ZZZZ']
target_tokens =  [item[1].split(',') for item in data]  #target_tokens[0] = ['AA', 'BB', ..., 'ZZ']

# define a function to generate token dictionary
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict    #input is a two-dimensional matrix, tokens is the ['AAAA', 'BBBB',...,'ZZZZ'], token is 'AAAA'

source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}  #we need the rev as {1:'AAAA', 2:'BBBB', ...}, which is for the sample inference
 
 
# add special tokens to every sequence
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens] #it's input of encoder
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens] #it's input of decoder
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens] #it's the ground truth of decoder

In [0]:
# padding and input preparation
source_max_len = max(map(len, encode_tokens))  #the size of source sequence, which is fixed
target_max_len = max(map(len, decode_tokens))   #the size of target sequence, which is fixed

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens] #tokens mean the sequence, padding enough <PAD> here!
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens] #map token in tokens into the number (sparse)
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens] 

In [5]:
# build our transformer model with the same embedding space
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=True,  # Use different embeddings for different languages
)

#without d2gpo
from keras.optimizers import Adam

#optimizer and compile
optimizer = Adam(lr=0.0001)
model.compile('adam', 'sparse_categorical_crossentropy')

#model fitting
train_start = 3750
train_end = 4850
model.fit(
    x=[np.array(encode_input)[train_start:train_end], np.array(decode_input)[train_start:train_end]],
    y=np.array(decode_output)[train_start:train_end],
    epochs=128,
    batch_size=16,
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/128
1100/1100 [==============================] - 6s 6ms/step - loss: 7.3154
Epoch 2/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.7262
Epoch 3/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.4685
Epoch 4/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.3597
Epoch 5/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.3143
Epoch 6/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.2887
Epoch 7/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.2655
Epoch 8/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.2432
Epoch 9/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.2155
Epoch 10/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.1867
Epoch 11/128
1100/1100 [==============================] - 2s 2ms/step - loss: 6.1546
Epoch 12/128
1100/1100 [==============================] - 2s 2ms/step - lo

In [0]:
#define the function to calculate the metric accuracy
def accuracy(i, k, temp):
    decoded = decode(
        model,
        encode_input[i],
        start_token=target_token_dict['<START>'],
        end_token=target_token_dict['<END>'],
        pad_token=target_token_dict['<PAD>'],
        top_k=3,
        temperature=1,
    )  
    predicted = (','.join(map(lambda x: target_token_dict_inv[x], decoded[1:-1]))).split(',')
    target = set(target_tokens[i])
    accuracy = 0
    for item in predicted:
        if item in target:
            accuracy += 1
    return accuracy / len(predicted)


In [7]:
#sensitivity analysis
import tqdm
topk_list = [1, 3, 5, 7]
temperature_list = [0.4, 1, 4, 10]

#analysis for topk
res_topk = []
for k in topk_list:
    res_topk.append([accuracy(i, k, 1) for i in tqdm.tqdm(range(train_end, 5040))])

#analysis for temperature
res_temp = []
for t in temperature_list:
    res_temp.append([accuracy(i, 3, t) for i in tqdm.tqdm(range(train_end, 5040))])

100%|██████████| 190/190 [00:37<00:00,  5.05it/s]


In [8]:
#model inference part
decoded = decode(
        model,
        encode_input[11],
        start_token=target_token_dict['<START>'],
        end_token=target_token_dict['<END>'],
        pad_token=target_token_dict['<PAD>'],
        top_k=3,
        temperature=1,
    )
predicted = (','.join(map(lambda x: target_token_dict_inv[x], decoded[1:-1]))).split(',')
target = target_tokens[11]
for i in zip(predicted, target):
    print(i)

('CSSE', 'CDMO')
('BCOV', 'PCYO')
('IIN', 'OCC')
('XSPA', 'CLWT')
('RLMD', 'BOOM')
('NTGN', 'WWR')
('NTGN', 'CTG')
('GERN', 'SNGX')
('SCKT', 'GERN')
('AOBC', 'CVV')
('AVID', 'PLUG')
('BPTH', 'GEOS')
('MTCH', 'SIGA')
('PDLI', 'OSIS')
('BNGO', 'CYAN')
('LX', 'FTEK')
('BTAI', 'IVAC')
('NMIH', 'AVDL')
('KNSA', 'ABMD')
('TTMI', 'TGA')
